In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, metrics
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
df = pd.DataFrame(columns=['x', 'y','steering','throttle','image'])
directory = 'images_labeled/'

images = list()
for f in os.listdir(directory):
    if os.path.isfile(directory+f):
        entry = f.split(",")
        for i in range(len(entry)-1):
            entry[i] = np.float32(entry[i])
            if i == 0 or i == 1:
                entry[i] = entry[i]/224
        entry[4] = directory+f
        df.loc[len(df.index)] = entry
df

In [ ]:
# data pipline
BATCH_SIZE = 32

# array of image files to read in parse_image
files = df["image"].tolist()

files = tf.constant(files)
# bounding_box = tf.constant(df[["x", "y"]].values.tolist())
steering = tf.constant(df["steering"].tolist())

# dataset
dataset = tf.data.Dataset.from_tensor_slices((files,steering))

dataset = dataset.shuffle(300)
ds_train = dataset.take(int(len(dataset)*.8)) # take first 80%
ds_validation = dataset.skip(int(len(dataset)*.8)).take(int(len(dataset)*.1)) # skip first 80%, take next 10%
ds_test = dataset.skip(int(len(dataset)*.9)) # skip first 90%, take rest

# image manipulation
# already normalizes pixel values
def parse_image(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    return image, label

# optimize pipeline
ds_train = ds_train.map(parse_image)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(parse_image)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
ds_validation = ds_validation.map(parse_image)
ds_validation = ds_validation.batch(BATCH_SIZE)
ds_validation = ds_validation.prefetch(tf.data.AUTOTUNE)

# batch iterator
iterator_train = iter(ds_train)
images_train, labels_train = iterator_train.get_next()
iterator_test = iter(ds_test)
images_test, labels_test = iterator_test.get_next()
iterator_validation = iter(ds_validation)
images_validation, labels_validation = iterator_validation.get_next()

In [25]:
# resnet50
res_input = layers.Input(shape=(244,244,3))
resnet50 = tf.keras.applications.resnet50.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=None, pooling='max', classes=1000
)(res_input)
resnet50 = layers.Flatten()(resnet50)
resnet50 = layers.Dense(4096, activation='relu')(resnet50)
resnet50 = layers.Dense(4096, activation='relu')(resnet50)
resnet50 = layers.BatchNormalization()(resnet50)
resnet50 = layers.Dense(2, activation='tanh')(resnet50)

resnet50 = keras.Model(res_input, resnet50)
resnet50.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 244, 244, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten_9 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 4096)              8392704   
_________________________________________________________________
dense_36 (Dense)             (None, 4096)              16781312  
_________________________________________________________________
batch_normalization_8 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_37 (Dense)             (None, 2)                 8194

In [ ]:
input_layer = layers.Input(shape=(224, 224, 3))
model = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_layer)
model = layers.MaxPooling2D((2,2), padding='same')(model)
model = layers.Conv2D(96, (3,3), activation='relu', padding='same')(model)
model = layers.MaxPooling2D((2,2), padding='same')(model)
model = layers.Conv2D(128, (3,3), activation='relu', padding='same')(model)
model = layers.MaxPooling2D((2,2), padding='same')(model)
model = layers.Conv2D(512, (3,3), activation='relu', padding='same')(model)
model = layers.MaxPooling2D((2,2), padding='same')(model)
model = layers.Flatten()(model)

model = layers.Dense(256, activation='relu')(model)
model = layers.Dense(128, activation='relu')(model)
model = layers.Dense(64, activation='relu')(model)
model = layers.Dense(2, activation='tanh')(model)
model = keras.Model(input_layer, model)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='MSE', metrics=["accuracy"])
model.fit(ds_train, epochs=100, validation_data=ds_validation)

In [ ]:
resnet50.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])
resnet50.fit(ds_train, epochs=1000, validation_data=ds_validation)

Epoch 1/1000
56/56 [==============================] - 46s 731ms/step - loss: 0.7817 - accuracy: 0.0958 - val_loss: 1.8224 - val_accuracy: 0.0000e+00
Epoch 2/1000
56/56 [==============================] - 40s 710ms/step - loss: 0.4546 - accuracy: 0.1922 - val_loss: 1.4068 - val_accuracy: 0.0000e+00
Epoch 3/1000
54/56 [===========================>..] - ETA: 1s - loss: 0.3214 - accuracy: 0.2130

In [ ]:
model.evaluate(ds_test)

In [ ]:
resnet50.evaluate(ds_test)

In [ ]:
# model.save("experiment_2")